# adaBoost Model

#### to do:

Add VIX and VIX return in the correlation_filter 

manage names in the features: spy_volume, spy_return, spy_close, etc

In [3]:
# Import appropriate modules

import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume 


# Generation of the Features Matrix X

In [4]:
# Ticker List: VIX must be in first position
ticker_list= ["^VIX", "spy", 'XLF', 'XLE',
              'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
              '^TNX', 'ZB=F', 'ZF=F', 'NQ=F','NKD=F',
              'LQD',
              'AAPL', 'AMZN', 'GE','MU','MSFT', 'BMY', 'FDX', 'GS','PLD','NVDA',   
              "tlt", "ief", 
              "FXI", "EZU", "EEM", "EFA", 'FEZ', "^GDAXI", '^FTSE','^HSI','^FCHI',
              "gld", "slv", "CL=F"]
    
#
# Energy Select Sector SPDR Fund (XLE)
# Financial Select Sector SPDR Fund (XLF)
# CAC 40 (^FCHI)
# Yen Denominated TOPIX Futures,D (TPY=F)
# HANG SENG INDEX (^HSI)
# FTSE 100 (^FTSE)
# DAX PERFORMANCE-INDEX (^GDAXI)
# SPDR EURO STOXX 50 ETF (FEZ)
# Nikkei/USD Futures,Dec-2021 (NKD=F)
# Nasdaq 100 Dec 21 (NQ=F)
# NVIDIA Corporation (NVDA)
# 'WTI CRUDE FUTURE' '261220.KS' -- eliminate
# Euro spot  'EURUSD'
# Treasury Yield 10 Years (^TNX) -- 1985
# U.S. Treasury Bond Futures,Dec- (ZB=F) - 2000
# Five-Year US Treasury Note Futu (ZF=F) - 2000
# American Funds U.S. Government Securities Fund Class C (UGSCX) - 2001
# PIMCO High Yield Municipal Bond Fund Class A (PYMAX) --eliminate
# Vanguard High-Yield Corporate Fund Investor Shares (VWEHX) -- 1985 --eliminate
# iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
# 13 Week Treasury Bill (^IRX) --eliminate
# Micron Technology, Inc. (MU)
# Microsoft Corporation (MSFT)
# Bristol-Myers Squibb Company (BMY)
# FEDEX CORP (FDX)
# The Goldman Sachs Group, Inc. (GS)
# Prologis, Inc. (PLD)


In [ ]:
#

In [ ]:
#


In [5]:
# Inclusion of security levels X1
def retrieve_close(close_prices_dict, ticker_list):
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2006-07-02', end_date='2021-10-02')
        close_prices_dict[ticker] = close_price
    return close_prices_dict



close_prices_dict = {}
close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
print("Completed retrieve of close prices")

close_prices_df= pd.DataFrame(close_prices_dict)

Processing Close ^VIX
Processing Close spy
Processing Close XLF
Processing Close XLE
Processing Close EURUSD=X
Processing Close GBPUSD=X
Processing Close AUDUSD=X
Processing Close BRLUSD=X
Processing Close DX-Y.NYB
Processing Close USDJPY=X
Processing Close ^TNX
Processing Close ZB=F
Processing Close ZF=F
Processing Close NQ=F
Processing Close NKD=F
Processing Close LQD
Processing Close AAPL
Processing Close AMZN
Processing Close GE
Processing Close MU
Processing Close MSFT
Processing Close BMY
Processing Close FDX
Processing Close GS
Processing Close PLD
Processing Close NVDA
Processing Close tlt
Processing Close ief
Processing Close FXI
Processing Close EZU
Processing Close EEM
Processing Close EFA
Processing Close FEZ
Processing Close ^GDAXI
Processing Close ^FTSE
Processing Close ^HSI
Processing Close ^FCHI
Processing Close gld
Processing Close slv
Processing Close CL=F
Completed retrieve of close prices


In [6]:
close_prices_df=close_prices_df.ffill(axis='rows')
close_prices_df=close_prices_df.bfill(axis='rows')

close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1=pd.concat([VIX,close_prices_component_df], axis=1)

In [7]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(
                                        security_returns_df, 
                                        min_corr=0.20, 
                                        key_column='^VIX', 
                                        eliminate_first_column=True 
)

X2=security_returns_component_df


#### OBSERVATION: It would be good to include VIX level and Return and check on both in case correlations changes in the future. As of now with some variables I check with VIX level, and others I check VIX return

In [8]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

volume_dict = {}
volume_dict = retrieve_volume(volume_dict, volume_list)
print("Completed retrieve of volume")

volume_df= pd.DataFrame(volume_dict)
volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3=volume_component_df

Processing Volume spy
Processing Volume XLF
Processing Volume XLE
Processing Volume EURUSD=X
Processing Volume GBPUSD=X
Processing Volume AUDUSD=X
Processing Volume BRLUSD=X
Processing Volume DX-Y.NYB
Processing Volume USDJPY=X
Processing Volume ^TNX
Processing Volume ZB=F
Processing Volume ZF=F
Processing Volume NQ=F
Processing Volume NKD=F
Processing Volume LQD
Processing Volume AAPL
Processing Volume AMZN
Processing Volume GE
Processing Volume MU
Processing Volume MSFT
Processing Volume BMY
Processing Volume FDX
Processing Volume GS
Processing Volume PLD
Processing Volume NVDA
Processing Volume tlt
Processing Volume ief
Processing Volume FXI
Processing Volume EZU
Processing Volume EEM
Processing Volume EFA
Processing Volume FEZ
Processing Volume ^GDAXI
Processing Volume ^FTSE
Processing Volume ^HSI
Processing Volume ^FCHI
Processing Volume gld
Processing Volume slv
Processing Volume CL=F
Completed retrieve of volume


In [9]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

not_to_include=['^VIX']


for ticker in ticker_list:
    
        if ticker in not_to_include:
            continue
    
        if (np.mean(abs(security_returns_df[ticker])))<=0.1:
            security_returns_df[ticker]=10*security_returns_df[ticker]
        
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], ticker, horizon=1, p=1, q=1, o=1)

        if (np.mean(abs(security_returns_df[ticker])))<=0.1:
            garch_series[ticker]=100*garch_series[ticker]
            
        
X4=garch_series
X4


starting series: spy
starting series: XLF
starting series: XLE
starting series: EURUSD=X
starting series: GBPUSD=X
starting series: AUDUSD=X
starting series: BRLUSD=X
starting series: DX-Y.NYB
starting series: USDJPY=X
starting series: ^TNX
starting series: ZB=F
starting series: ZF=F
starting series: NQ=F
starting series: NKD=F


/Users/Paola/Applications/anaconda3/envs/dev/lib/python3.7/site-packages/arch/univariate/base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.05177. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning


starting series: LQD
starting series: AAPL
starting series: AMZN
starting series: GE
starting series: MU
starting series: MSFT
starting series: BMY
starting series: FDX
starting series: GS
starting series: PLD
starting series: NVDA
starting series: tlt
starting series: ief
starting series: FXI
starting series: EZU
starting series: EEM
starting series: EFA
starting series: FEZ
starting series: ^GDAXI
starting series: ^FTSE
starting series: ^HSI
starting series: ^FCHI
starting series: gld
starting series: slv
starting series: CL=F


,spy,XLF,XLE,EURUSD=X,GBPUSD=X,AUDUSD=X,BRLUSD=X,DX-Y.NYB,USDJPY=X,^TNX,...,EEM,EFA,FEZ,^GDAXI,^FTSE,^HSI,^FCHI,gld,slv,CL=F
Date,,,,,,,,,,,,,,,,,,,,,
2006-07-04,0.006692,0.000080,0.000132,0.004522,0.004817,0.004917,0.015749,0.003995,0.005059,0.000062,...,0.000202,0.010501,0.000122,0.011578,0.008011,0.008076,0.009917,0.013323,0.000227,0.000133
2006-07-05,0.007151,0.000090,0.000128,0.004539,0.004862,0.005003,0.015514,0.003929,0.004967,0.000062,...,0.000227,0.012812,0.000146,0.013629,0.008744,0.008175,0.011203,0.013095,0.000223,0.000128
2006-07-06,0.006764,0.000086,0.000124,0.004499,0.004788,0.004959,0.014738,0.003900,0.004855,0.000065,...,0.000218,0.012280,0.000140,0.013013,0.008388,0.008149,0.010641,0.012959,0.000221,0.000125
2006-07-07,0.007414,0.000085,0.000129,0.004472,0.004840,0.004925,0.014039,0.003912,0.006209,0.000069,...,0.000221,0.011880,0.000137,0.012489,0.008057,0.007972,0.010201,0.012741,0.000220,0.000138
2006-07-10,0.007001,0.000083,0.000125,0.004582,0.004860,0.005544,0.013301,0.004024,0.005957,0.000068,...,0.000211,0.011379,0.000131,0.011942,0.007748,0.007909,0.009711,0.012598,0.000219,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.008687,0.000120,0.000175,0.003282,0.004207,0.006098,0.009765,0.002941,0.004153,0.000273,...,0.000127,0.009676,0.000121,0.011188,0.007166,0.015791,0.010372,0.008502,0.000153,0.000165
2021-09-28,0.013708,0.000138,0.000169,0.003275,0.004161,0.005912,0.010005,0.003007,0.004038,0.000271,...,0.000132,0.012998,0.000160,0.014098,0.007244,0.015364,0.014307,0.008526,0.000151,0.000157
2021-09-29,0.012732,0.000130,0.000163,0.003241,0.004806,0.006128,0.009852,0.003185,0.004139,0.000266,...,0.000132,0.012432,0.000154,0.013455,0.006993,0.014883,0.013530,0.008412,0.000164,0.000152


In [10]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5=returns_squared_component_df
X5

,spy,XLF,XLE,GBPUSD=X,BRLUSD=X,DX-Y.NYB,^TNX,ZF=F,NQ=F,NKD=F,...,EZU,EEM,EFA,FEZ,^GDAXI,^FTSE,^HSI,^FCHI,gld,slv
Date,,,,,,,,,,,,,,,,,,,,,
2006-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-04,0.000000,0.000000,0.000000,0.000197,0.000903,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000816,0.000002,0.000672,0.000043,0.000000,0.000000
2006-07-05,0.003263,0.008976,0.002055,0.002530,0.013049,0.000000,0.000000,0.000000,0.043742,0.018885,...,0.025416,0.096398,0.036164,0.041579,0.032562,0.009320,0.003868,0.015767,0.002648,0.006255
2006-07-06,0.000848,0.000952,0.000678,0.000095,0.001901,0.000853,0.006457,0.000186,0.000429,0.000042,...,0.006378,0.041738,0.011121,0.018073,0.015412,0.011802,0.011416,0.008417,0.006922,0.012649
2006-07-07,0.004242,0.002422,0.018421,0.005734,0.002914,0.001879,0.010448,0.000388,0.008883,0.027276,...,0.003874,0.035976,0.002120,0.004986,0.000572,0.000003,0.000131,0.000658,0.003830,0.055803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.000818,0.018511,0.126450,0.001518,0.004325,0.000029,0.027022,0.000068,0.006552,0.000044,...,0.000144,0.004223,0.000039,0.000018,0.000736,0.000285,0.000047,0.000352,0.000433,0.012334
2021-09-28,0.040610,0.027293,0.001166,0.000568,0.009814,0.001744,0.113520,0.000058,0.080085,0.029835,...,0.056871,0.016675,0.047554,0.065690,0.043634,0.002498,0.014510,0.047145,0.009441,0.005838
2021-09-29,0.000283,0.000062,0.000000,0.013532,0.004697,0.003695,0.002082,0.000058,0.000287,0.003110,...,0.002428,0.008679,0.000526,0.002954,0.005858,0.012989,0.004432,0.006965,0.002029,0.159538


### Inclusion of Google Trends

In [11]:
# Upload of Google Tremds -- X6
keywords=['banking', "consumer", "depression", "gdp", "inflation",
          'unemployment', 'liquidity','cci', 'vix_word'] #'jobless_claims_pao', 
google_trends_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Google_trends/{keyword}.csv"
    print(file_path)
    trend=pd.read_csv(Path(file_path),
                      index_col= 'Daily', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    #print(trend)
    google_trends_df=pd.concat([google_trends_df, trend], axis=1)
    #print(google_trends_df)

google_trends_df

./Resources/Google_trends/banking.csv
./Resources/Google_trends/consumer.csv
./Resources/Google_trends/depression.csv
./Resources/Google_trends/gdp.csv
./Resources/Google_trends/inflation.csv
./Resources/Google_trends/unemployment.csv
./Resources/Google_trends/liquidity.csv
./Resources/Google_trends/cci.csv
./Resources/Google_trends/vix_word.csv


,Banking: (United States),Consumer: (United States),Depression: (United States),GDP: (United States),inflation,unemployment: (United States),Liquidity: (United States),cci,VIX:(United States),f
Daily,,,,,,,,,,
2004-01-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-02,33,54,81,0.0,43.0,0.0,0.0,0.0,0.0,1.0
2004-01-03,97,22,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-04,27,22,100,0.0,53.0,0.0,0.0,0.0,0.0,1.0
2004-01-05,64,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2021-10-04,78,95,48,23.0,38.0,16.0,16.0,0.0,0.0,1.0
2021-10-05,63,65,26,24.0,78.0,19.0,25.0,0.0,15.0,1.0
2021-10-06,72,87,51,23.0,32.0,12.0,66.0,51.0,29.0,1.0


In [12]:
# Working on preparing Google-trends data

# Unifying google dates with VIX
minimum_date=vix.index.min()
maximum_date=vix.index.max()

google_trends_df=google_trends_df.loc[minimum_date:maximum_date,:]
#print(google_trends_df.iloc[0,:])

vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)
vix_google_trends_df.isna().sum()

#print(vix_google_trends_df.head())

#vix_google_trends_df=vix_google_trends_df.fillna(0)
#vix_google_trends_df
#vix_google_trends_df.loc[vix_google_trends_df['^VIX'].isna(),['^VIX','Banking: (United States)']]

# We will drop Saturday Sunday, but we would like to average Fri-Sat-Sun and reset the value of Friday
vix_google_trends_df=vix_google_trends_df.dropna()
google_trends_df=vix_google_trends_df.iloc[:,1:]
#google_trends_df.isna().sum()

In [13]:
# Filtering by correlation X6

google_trends_component_df = correlation_filter(
                                vix_google_trends_df, 
                                min_corr=0.05, 
                                key_column='^VIX', 
                                eliminate_first_column=True)

X6=google_trends_component_df

# We will interpolate so we can fill the missing data only on Google Trends
pro_interpolation_of_X6=pd.concat([vix, X6], axis=1)
pro_interpolation_of_X6=pro_interpolation_of_X6.interpolate(method="polynomial", order=2, axis=0)
pro_interpolation_of_X6
X6 = pro_interpolation_of_X6.iloc[:,1:]
X6.shape

(3980, 3)

# INCLUSION OF LAGGED SERIES

In [14]:
# Inclusion of different lags of data from the X -- up to n_lag



# Inclusion of Economic and Financial Series X7

In [15]:
#Economic Series
# Upload of csv files -- X7
keywords=['JobClaimsWeeklySeries', 'vix_put_call_ratio']
economic_series_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Economic_and_financial_Series/{keyword}.csv"
    print(file_path)
    new_series=pd.read_csv(Path(file_path),
                      index_col= 'DATE', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    if keyword=='JobClaimsWeeklySeries':
        new_series=new_series.shift(-1, freq='D')
    print(new_series)
    # Adjustment due to weekend data. We are going to assign data on the weekends to Friday, since are going to be 
    # consider for the the prediction of Monday
    economic_series_df=pd.concat([economic_series_df, new_series], axis=1)
    #print(economic_series_df.tail())

economic_series_df
economic_series_change_df = economic_series_df.pct_change().add_suffix('_change')

economic_series_df.loc[:,:].tail(20)

./Resources/Economic_and_financial_Series/JobClaimsWeeklySeries.csv
              ICSA
DATE              
1967-01-06  208000
1967-01-13  207000
1967-01-20  217000
1967-01-27  204000
1967-02-03  216000
...            ...
2021-09-03  312000
2021-09-10  335000
2021-09-17  351000
2021-09-24  364000
2021-10-01  326000

[2857 rows x 1 columns]
./Resources/Economic_and_financial_Series/vix_put_call_ratio.csv
            VIX Put/Call Ratio
DATE                          
2006-02-24                1.18
2006-02-27                0.01
2006-02-28                1.75
2006-03-01                0.70
2006-03-02                0.18
...                        ...
2019-09-30                0.81
2019-10-01                0.97
2019-10-02                0.56
2019-10-03                0.46
2019-10-04                0.49

[3427 rows x 1 columns]


,ICSA,VIX Put/Call Ratio
DATE,,
2021-05-21,405000.0,NaN
2021-05-28,388000.0,NaN
2021-06-04,374000.0,NaN
2021-06-11,418000.0,NaN
2021-06-18,416000.0,NaN
2021-06-25,368000.0,NaN
2021-07-02,386000.0,NaN
2021-07-09,368000.0,NaN
2021-07-16,424000.0,NaN


In [16]:
# Working on indexes available

# Changes of columns that are on a weekend - concat with vix to add week days
vix_economic= pd.concat([vix,economic_series_df,economic_series_change_df ],axis=1)
vix_economic['VIX Put/Call Ratio']= vix_economic['VIX Put/Call Ratio'].fillna(0)

# Applying interpolation to appropiate columns. Levels: interpolation, changes: zeros
vix_economic.loc[:,economic_series_df.columns]=vix_economic.loc[:,economic_series_df.columns].interpolate(method="polynomial", order=2, axis=0)
vix_economic.loc[:,economic_series_change_df.columns]=vix_economic.loc[:,economic_series_change_df.columns].fillna(0)

#print(vix_economic)

#Filtering for available dates
economic_series_ready_df = vix_economic.loc[minimum_date:maximum_date,:]
economic_series_ready_df = economic_series_ready_df.iloc[:,1:]

X7=economic_series_ready_df

In [17]:
# SPY volatility on varios tracks X8

close_price_spy_df=retrieve_yahoo_close('spy', start_date='2005-01-01', end_date='2021-10-02')
spy_returns_df=close_price_spy_df.pct_change()

spy_volatility_30_days = spy_returns_df.rolling(window=30).std()
spy_volatility_60_days = spy_returns_df.rolling(window=60).std()
spy_volatility_90_days = spy_returns_df.rolling(window=90).std()
spy_volatility_260_days = spy_returns_df.rolling(window=260).std()
spy_volatility_360_days = spy_returns_df.rolling(window=360).std()
spy_volatility_10_days = spy_returns_df.rolling(window=10).std()
spy_volatility_20_days = spy_returns_df.rolling(window=20).std()
spy_volatility_180_days = spy_returns_df.rolling(window=180).std()
spy_volatility_200_days = spy_returns_df.rolling(window=200).std()
spy_volatility_120_days = spy_returns_df.rolling(window=120).std()

X8=pd.concat([security_returns_df['spy'],
              spy_volatility_30_days ,
              spy_volatility_60_days,
              spy_volatility_90_days,
              spy_volatility_260_days,
              spy_volatility_360_days,
              spy_volatility_10_days,
               spy_volatility_20_days,
               spy_volatility_180_days,
               spy_volatility_200_days,
              spy_volatility_120_days],
             axis=1
                )
X8=X8.loc[minimum_date:maximum_date,:]
X8=X8.ffill()
X8=X8.iloc[:,1:]
X8.shape


Processing Close spy


(3980, 10)

In [18]:
# Inclusion of Technical Indicators


# GENERATION OF THE FEATURE MATRIX **X**

In [19]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X6, X7, X8], axis=1)
print(XY.shape)

XY.dropna(subset = ['VIX', 'VIX_ret'])
print(XY.shape)

# Interpolation is not applied to numerical variables. We are just going to drop those.
print(f"XY.shape: {XY.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X1.shape: {X1.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X2.shape: {X2.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X3.shape: {X3.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X4.shape: {X4.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X5.shape: {X5.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X6.shape: {X6.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X7.shape: {X7.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X8.shape: {X8.shape}, {XY.index.min()}, {XY.index.max()} ")

XY.isna().head(40)
XY.isna().sum().tail(40)
#XY=XY.dropna()
XY.shape


(3980, 159)
(3980, 159)
XY.shape: (3980, 159), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X1.shape: (3980, 21), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X2.shape: (3980, 28), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X3.shape: (3980, 21), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X4.shape: (3979, 39), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X5.shape: (3980, 33), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X6.shape: (3980, 3), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X7.shape: (3980, 4), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X8.shape: (3980, 10), 2006-07-03 00:00:00, 2021-10-01 00:00:00 


(3980, 159)

In [20]:
# Set the Signal column
threshold=0.030
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret'] >= threshold), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
print(XY["Signal"].head())    
XY["Signal"].value_counts()
XY.loc[XY["Signal"]==1, 'VIX_ret']
#XY.shape  ## 3981

2006-07-03    0.0
2006-07-04    0.0
2006-07-05    1.0
2006-07-06    0.0
2006-07-07    0.0
Freq: B, Name: Signal, dtype: float64


2006-07-05    0.084291
2006-07-12    0.102740
2006-07-13    0.227743
2006-07-17    0.032687
2006-07-20    0.042444
                ...   
2021-09-10    0.114362
2021-09-17    0.113430
2021-09-20    0.235464
2021-09-27    0.056901
2021-09-28    0.239339
Name: VIX_ret, Length: 1048, dtype: float64

In [21]:
vix_ret=vix.pct_change()
vix_ret[vix_ret>=threshold].index
vix_ret.shape


# How many values of the vix we missed due to missing data on other series
compare=pd.concat([XY.loc[XY["Signal"]==1, 'VIX_ret'],vix_ret[vix_ret>=threshold] ], axis=1)
missing_dates=compare.loc[compare["VIX_ret"]!=compare["^VIX"]]
missing_dates=missing_dates.index
missing_dates
vix[missing_dates]

Series([], Name: ^VIX, dtype: float64)

In [22]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
y

2006-07-03    0.0
2006-07-04    0.0
2006-07-05    1.0
2006-07-06    0.0
2006-07-07    0.0
             ... 
2021-09-27    1.0
2021-09-28    1.0
2021-09-29    0.0
2021-09-30    0.0
2021-10-01    0.0
Freq: B, Name: Signal, Length: 3980, dtype: float64

In [23]:
# Define features set X by selecting all columns but Signal
X = XY.drop(columns=["Signal"]).shift().dropna()


# Review the features DataFrame
X.head()

,VIX,VIX_ret,spy,XLF,XLE,USDJPY=X,NKD=F,LQD,GE,BMY,...,spy,spy,spy,spy,spy,spy,spy,spy,spy,spy
2007-07-06,15.480000,0.037533,114.451202,22.203014,46.896809,122.949997,18235.0,59.342045,194.811691,19.049101,...,0.008297,0.006848,0.008302,0.006463,0.006654,0.007756,0.009343,0.006656,0.006471,0.007525
2007-07-09,14.720000,-0.049096,115.052895,22.312513,47.534676,123.360001,18205.0,59.227097,194.508377,18.834181,...,0.008350,0.006838,0.007155,0.006460,0.006658,0.007736,0.008323,0.006664,0.006479,0.007510
2007-07-10,15.160000,0.029891,115.143135,22.221266,47.979858,123.370003,18235.0,59.347828,195.216080,18.714794,...,0.008161,0.006824,0.007089,0.006460,0.006645,0.006883,0.007872,0.006658,0.006474,0.007506
2007-07-11,17.570000,0.158971,113.503616,21.752871,47.554592,121.290001,18080.0,59.756054,191.576691,18.464069,...,0.008567,0.007086,0.007258,0.006524,0.006682,0.008363,0.008566,0.006751,0.006554,0.007629
2007-07-12,16.639999,-0.052931,114.308304,21.825867,47.734005,122.269997,18125.0,59.485786,193.093124,18.875973,...,0.008641,0.007042,0.007131,0.006509,0.006660,0.007584,0.008287,0.006767,0.006557,0.007643


In [ ]:
X.corr()

In [24]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=50)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]


In [25]:
# Create a StandardScaler instance
scaler =  StandardScaler() # MinMaxScaler() #

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [26]:
# Instance AdaBoost
# Initiate the model instance
adaboost_model=AdaBoostClassifier(n_estimators=120)
adaboost_model


AdaBoostClassifier(n_estimators=120)

In [27]:
# Fit the model 
adaboost_model =adaboost_model.fit(X_train_scaled, y_train)

pred_adaboost=adaboost_model.predict(X_test_scaled)

ValueError: Found input variables with inconsistent numbers of samples: [983, 1088]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
adaboost_report=classification_report(y_test, pred_adaboost)

# Print the classification report
print("         AdaBoost Classification Report")
print(adaboost_report)


In [ ]:
# Number of estimators? 

for n in range (50,200, 10):
    # Instance AdaBoost
    # Initiate the model instance
    adaboost_model=AdaBoostClassifier(n_estimators=n)

    # Fit the model 
    adaboost_model =adaboost_model.fit(X_train_scaled, y_train)
    pred_adaboost=adaboost_model.predict(X_test_scaled)
    print (n)
    # Use a classification report to evaluate the model using the predictions and testing data
    adaboost_report=classification_report(y_test, pred_adaboost)

    # Print the classification report
    print("         AdaBoost Classification Report")
    print(adaboost_report)
#120 highest 1-recall
#150 good overall accuracy, but lower 1-recall
